In [ ]:
import os
import sys
import shutil
import platform
import numpy as np
import pandas as pd
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import Image
from matplotlib.patches import Rectangle
from matplotlib.colors import Normalize
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
sys.path.append(os.path.join("..","..","pyemu"))
sys.path.append(os.path.join("..","..","flopy"))
import pyemu

This is an early demo of how the outputs from PESTPP-MOU can be used for decision support. Subject to change

In [ ]:
pst = pyemu.Pst(os.path.join("henry.pst"))
par = pst.parameter_data
par = par.loc[par.pargp=="dv_pars",:].copy()
rate_pars = par.loc[par.parnme.apply(lambda x: not x.startswith("ar")),"parnme"].to_list()
rate_pars

In [ ]:
df_org = pd.read_csv(os.path.join("henry.75.archive.dv_pop.csv"))
keep_cols = [c for c in df_org.columns if c.startswith("ar")]
keep_cols.extend(rate_pars)
df_org = df_org.loc[:,keep_cols]
df_org.loc[:, rate_pars] *= -1.

tot = df_org.loc[:, rate_pars].sum(axis=1)

In [ ]:
def plot(pump_min,concen_min,rch_max):
    

    cmap = plt.get_cmap("viridis")
    norm = Normalize(0.0,35.0)    
        

    # only show solutions with some min amount of pumping
    df_gen = df_org.loc[tot > pump_min]
    # and only show solutions using concen > some % seawater
    df_gen = df_gen.loc[df_gen.ar_concen >= concen_min]
    df_gen = df_gen.loc[df_gen.ar_rate <= rch_max]

    
    
        
    fig,ax = plt.subplots(1,1,figsize=(10,6))
    #ax = axes[0]
    axt = plt.twinx()
    if df_gen.shape[0] == 0:
        ax.text(0.5,0.5,"No Feasible Solutions",ha="center",va="center",fontsize=10)
        ax.set_xlim(0,1)
        ax.set_ylim(0,1)
        return
    
    #dmn = df_gen.ar_dist.min()
    #dmx = df_gen.ar_dist.max()
    
    xlim = (0,160)
    ylim = (0,3.5)
    
    for d,w,r,c,r1,r2,r3 in zip(df_gen.ar_dist,df_gen.ar_width,df_gen.ar_rate,
                       df_gen.ar_concen,df_gen.loc[:,rate_pars[0]],
                                df_gen.loc[:,rate_pars[1]],
                                df_gen.loc[:,rate_pars[2]]):
        r = Rectangle((d,0),w,r,facecolor=cmap(c/35.0),edgecolor="none",alpha=0.5)
        ax.add_patch(r)
        mpt = d + (w/2.)
        #axes[1].plot([mpt,mpt],[0,r1],color=cmap(c/35.0))
        #axes[2].plot([mpt, mpt], [0, r2], color=cmap(c / 35.0))
        #axes[3].plot([mpt, mpt], [0, r3], color=cmap(c / 35.0))
        axt.plot([mpt,mpt],[0,-1*(r1+r2+r3)],c="0.5")
        
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_xlabel("column",fontsize=10)
    ax.set_ylabel("recharge flux rate",fontsize=10)
    axt.set_xlim(xlim)
    axt.set_ylim(-3.5,0)
    axt.set_ylabel("total extraction rate", fontsize=10)
    cb = plt.colorbar(plt.cm.ScalarMappable(norm=norm,cmap=cmap),orientation="horizontal")
    cb.set_label("recharge concentration",fontsize=10)
    ax.set_title("{0} feasible nondom solutions (bar width=basin location, height=rch rate, color=rch concen)".format(df_gen.shape[0]),fontsize=10)
#     for i,ax in enumerate(axes[1:]):
#         ax.set_ylim(ylim)
#         ax.set_xlim(xlim)
#         ax.set_title("pumping well {0}".format(i+1),fontsize=10)
#         ax.set_ylabel("pumping rate",fontsize=10)


    plt.tight_layout()
    



In [ ]:
pump_slider = widgets.FloatSlider(description="min ext rate", 
                                           continuous_update=False, min=0.0,max=2.0,value=1.0)
concen_slider = widgets.FloatSlider(description="min rch conc", 
                                           continuous_update=False, min=0.01,max=35.0,value=3.5)
rate_slider = widgets.FloatSlider(description="max rch rate", 
                                           continuous_update=False, min=0.0,max=2.0,value=2.0)

In [ ]:
if False:
    import flopy
    ucn = flopy.utils.HeadFile(os.path.join(m_d,"trans.ucn"),text="concentration").get_data()
    fig,ax = plt.subplots(1,1,figsize=(10,3))
    cb = ax.imshow(ucn[:,0,:],vmin=0.0,vmax=35.0)

    #c = plt.colorbar(cb)
    #c.set_label("concentration",fontsize=15)
    ax.scatter([60,60,60],[10,20,30],c='r',marker='o',s=50)
    ax.text(62,20,"pumping wells",color="r",fontsize=15,ha="left",va="center", rotation=90)
    ax.plot([90,100],[0,0],"r-",lw=10)
    ax.text(95,2,"possible\nrecharge\nbasin",color="r",fontsize=15,ha="center",va="top")
    ax.plot([0,40],[0,0],"r-",lw=10)
    ax.text(20,4,"areal recharge",color="r",fontsize=15,ha="center",va="center")
    ax.plot([159,159],[0,40],"r-",lw=10)
    ax.text(155,20,"coastal boundary",color="r",fontsize=15,ha="center",va="center",rotation=90)

    ax.set_title("Henry-ish model domain",fontsize=15)
    ax.set_ylim(40,0)
    ax.set_ylabel("layer",fontsize=15)
    _ = ax.set_xlabel("column",fontsize=15)
    plt.tight_layout()
    plt.savefig("henry_domain.png")




In [ ]:
Image("henry_domain.png",width=700,unconfined=True)

The purpose of this analysis is to design an optimal recharge basin and combination extraction well usage scheme to maximize water being withdrawn for potable use.

Decision variables:
- rates at three extraction wells
- distance from left edge to start of recharge basin
- width of recharge basin
- recharge basin rate
- recharge basin concentration

Constraints:
- water removed by three extraction wells has concentration less 0.01 g/l

Objectives:
- minimize recharge basin width (prefer a small basin)
- minimize recharge basin rate (prefer to use less water)
- maximize recharge basin concentration (prefer to use shitty water for recharge basin)
- maximize extraction well rates (well, yeah!)

Use the sliders below to enforce limits on the objectives and see what possible solutions exist

In [ ]:
interact(plot, pump_min=pump_slider, concen_min=concen_slider, rch_max=rate_slider,);
Image("henry_domain.png",width=700,unconfined=True)